### Create Database Connection and Set Configurations

In [1]:
%load_ext sql
# call jupysql
%sql sqlite:///chinook.db
# connect to DB

In [2]:
%config SqlMagic.feedback = False
# if false prevent showing feedback from duckdb
%config SqlMagic.displaycon = False
# if false prevent showing "Running query in 'sqlite:///chinook.db'" in each cell
%config SqlMagic.displaylimit = 100
# limit number of rows printed

### Overview of the Data

In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### Introduction
The Chinook record store signed a deal to buy albums with a new record label that specializes in artists from the USA. 

There are four albums that currently don't have tracks in the store, namely: 

- Artist Name : Regal, Genre : Hip-Hop
- Artist Name : Red Tone, Genre : Punk
- Artist Name : Meteor and the Girls, Genre : Pop
- Artist Name : Slim Jim Bites, Genre : Blues

The store will only chose three out of four albums. The record label has also given some money to the store to advertise the chosen albums in the USA. 

### Business Questions 
1.Whose album the store should purchase, based on number of tracks sold in the USA? 

In [8]:
%%sql

WITH base AS (
  
SELECT t.genre_id,g.name,
       SUM(il.quantity) AS total_sold
  FROM invoice AS i
  JOIN invoice_line AS il  
    ON i.invoice_id=il.invoice_id
  JOIN track AS t
    ON il.track_id=t.track_id
  JOIN genre AS g
    ON t.genre_id=g.genre_id
 WHERE i.billing_country = 'USA'
 GROUP BY t.genre_id ),

track_sold AS (SELECT genre_id,name,
       total_sold, 
       ROUND(CAST (total_sold AS FLOAT)*100/(SELECT SUM(total_sold)
                                               FROM base),2)||'%' AS total_percentage
  FROM base
 ORDER BY total_sold DESC)

SELECT ts.genre_id,ts.name,ts.total_sold,ts.total_percentage,ai.album_id
 FROM track_sold AS ts
  JOIN track AS t
   ON ts.genre_id=t.genre_id
 JOIN album AS ai
   ON t.album_id=ai.album_id
  GROUP BY ts.genre_id
  ORDER BY total_sold DESC;

genre_id,name,total_sold,total_percentage,album_id
1,Rock,561,53.38%,1
4,Alternative & Punk,130,12.37%,11
3,Metal,124,11.8%,9
14,R&B/Soul,53,5.04%,115
6,Blues,36,3.43%,20
23,Alternative,35,3.33%,260
9,Pop,22,2.09%,29
7,Latin,22,2.09%,21
17,Hip Hop/Rap,20,1.9%,184
2,Jazz,14,1.33%,8


#### Visualization on the most sold genre in Chinook Store : 

https://public.tableau.com/app/profile/novita.eliana/viz/ChinookRecordStore-CustomerPreferences/ChinookRecordStore-CustomerPreferences

#### INSIGHT 
Based on the data above, out of four albums that currently don't have tracks in the store,  the store should purchase albums from: Red Tone with genre Punk, Meteor and the Girls with genre Pop, Slim Jim Bites with Genre Blues, considering the number of sales of the three albums combined is accounted for 17.89% of total sales. However, it's important to notice that, all of those three albums are not the highest-selling genre. The store should consider buying albums with rock genre since it's accounted for 53% of the total sales in the USA. 

### Introduction

In the Chinook store, a customer can make purchases with several conditions:

1.purchase a whole album 

2.purchase a collection of one or more individual tracks

3.The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually

4.When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately

5.Ignore these two edge cases: 

- 5.1 Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks

- 5.2 Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase
                
### Business Questions
2. Find out what number of invoices and percentage of invoices are individual tracks vs whole albums, to help store management purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

In [13]:
%%sql 
WITH track_purchase AS (

 SELECT  i.invoice_id,t.album_id,COUNT(t.track_id) AS number_of_track_bought
   FROM invoice_line AS i
   JOIN track AS t
     ON i.track_id=t.track_id
   JOIN album AS a
     ON t.album_id=a.album_id
    GROUP BY i.invoice_id,t.album_id
), 

album_track AS( 
  SELECT a.album_id,COUNT(t.track_id) AS number_of_track
    FROM album AS a
    JOIN track AS t
      ON a.album_id=t.album_id
    GROUP BY a.album_id
),

 final_table AS(
     SELECT tp.invoice_id,tp.album_id,number_of_track_bought,number_of_track
      FROM track_purchase tp
 LEFT JOIN album_track AS at
        ON tp.album_id=at.album_id),
    
status_table AS (SELECT *, 
           (CASE 
            WHEN number_of_track_bought=number_of_track THEN 'Whole Album'
             ELSE 'Single Track'
              END) AS purchase_status
      FROM final_table)

SELECT purchase_status,COUNT(*) AS total_invoice
  FROM status_table
 GROUP BY purchase_status;

purchase_status,total_invoice
Single Track,2870
Whole Album,207


#### Visualization on number of invoice of individual tracks vs whole albums : 

https://public.tableau.com/app/profile/novita.eliana/viz/ChinookRecordStore-CustomerPreferences/ChinookRecordStore-CustomerPreferences

#### INSIGHT: 
The sales of individual tracks is thirteen times bigger than the sales of the whole album.  

### Introduction
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase.

### Business Question 
3.Which employees have top sales performance?

In [4]:
%%sql 

WITH base AS (
SELECT c.support_rep_id,ROUND(SUM(i.total),2) AS total_sales
  FROM invoice AS i
  JOIN customer AS c 
    ON i.customer_id=c.customer_id
  GROUP BY c.support_rep_id)

SELECT e.first_name,e.last_name,e.title,e.reports_to,b.*,e.country
  FROM base AS b
  JOIN employee AS e
    ON b.support_rep_id=e.employee_id;

first_name,last_name,title,reports_to,support_rep_id,total_sales,country
Jane,Peacock,Sales Support Agent,2,3,1731.51,Canada
Margaret,Park,Sales Support Agent,2,4,1584.0,Canada
Steve,Johnson,Sales Support Agent,2,5,1393.92,Canada


#### Visualization on best performing employees : 

https://public.tableau.com/app/profile/novita.eliana/viz/ChinookRecordStore-SalesPerformance/ChinookRecordStore-Sales

#### INSIGHT : 
Based on data above, Jane Peacock has the highest sales performance. However, when being compared with the second highest sales performance, the difference in total sales only 147 points. 

### Introduction

The Chinook store would like to know the analysis of sales data for customers from each different country. In doing the analysis, there are some guidances to be followed: 

- use the country value from the customers table, and ignore the country from the billing address in the invoice table.

- there are a number of countries with only one customer, you should group these customers as "Other" in the analysis and force the ordering of "Other" to last in the analysis.

### Business Question

3.Analyze sales data for customers in each different country on the: 

- 3.1 total number of customer
- 3.2 total value of sales
- 3.3 average value of sales per customer
- 3.4 average order value

3.1 total number of customer

In [20]:
%%sql 
SELECT c.country,
       COUNT(DISTINCT c.customer_id) AS total_cust,
       ROUND (SUM(i.total),2) AS total_spent
  FROM invoice AS i
  JOIN customer AS c
    ON i.customer_id=c.customer_id
 GROUP BY c.country;

country,total_cust,total_spent
Argentina,1,39.6
Australia,1,81.18
Austria,1,69.3
Belgium,1,60.39
Brazil,5,427.68
Canada,8,535.59
Chile,1,97.02
Czech Republic,2,273.24
Denmark,1,37.62
Finland,1,79.2


3.2 total value of sales

In [28]:
%%sql 

WITH base AS (SELECT c.country,
       COUNT(DISTINCT c.customer_id) AS total_cust,
       ROUND (SUM(i.total),2) AS total_spent
  FROM invoice AS i
  JOIN customer AS c
    ON i.customer_id=c.customer_id
 GROUP BY c.country),

base2 AS (SELECT *, 
       (CASE 
          WHEN total_cust=1 THEN 'Others'
          ELSE country
           END) AS new_country_category
  FROM base),
  
base3 AS (SELECT new_country_category,
           SUM(total_spent) AS values_of_sales
      FROM base2
      GROUP BY new_country_category) 

  SELECT *
    FROM (
          SELECT *,(CASE 
                  WHEN new_country_category='Others' THEN 1
                  ELSE 0
                   END) AS sort
            FROM base3 ) 
                
     ORDER BY sort ASC;

new_country_category,values_of_sales,sort
Brazil,427.68,0
Canada,535.59,0
Czech Republic,273.24,0
France,389.07,0
Germany,334.62,0
India,183.15,0
Portugal,185.13,0
USA,1040.49,0
United Kingdom,245.52,0
Others,1094.94,1


#### Visualization on total value of sales per country : 

https://public.tableau.com/app/profile/novita.eliana/viz/ChinookRecordStore-SalesPerformance/ChinookRecordStore-Sales

3.3 average value of sales per customer

In [29]:
%%sql

SELECT customer_id,ROUND(AVG(total),2) AS average_sales_value
  FROM invoice
  GROUP BY customer_id
  ORDER BY average_sales_value DESC;

customer_id,average_sales_value
3,11.11
6,10.73
29,10.15
18,9.9
37,9.41
27,9.35
16,9.28
42,9.09
24,8.91
50,8.91


3.4 average order value

In [32]:
%%sql 

WITH base AS ( SELECT SUM(total) AS total_sales,COUNT(invoice_id) AS total_order
  FROM invoice)

SELECT ROUND (total_sales/total_order,2) AS company_average_order_value 
  FROM base;

company_average_order_value
7.67
